# Quickstart

This notebook will cover all the basic and most useful functionality available to get a user up and running as fast as possible.

We will skim over these topics:

- installation
- function definition
- remote connection
- dataset creation
- creating runs
- running and retrieving results

### Installation

Installation can be done via a pip install:

`pip install remotemanager` for the most recent stable version.

However if you would like the bleeding edge version, you can clone the `devel` branch of the git [repository](https://gitlab.com/ljbeal/remotemanager):

`git clone --branch devel`

`cd remotemanager && pip install .`

### Function Definition

remotemanager executes user defined python functions at the location of choice. Below is a basic function example which will serve our purposes for this guide

.. note::
    The function must stand by itself when running, so any imports or necessary functionality should be contained within

In [11]:
!pip install -U remotemanager

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 1.6 MB/s eta 0:00:00a 0:00:01


In [1]:
def multiply(a, b):
    import time
    
    time.sleep(1)
    
    return a * b

### Remote Connection

This function would run just fine on any workstation, however imagine that the function is something significantly more demanding. We would need to connect to some more powerful resources for this.

remotemanager provides the powerful URL module for this purpose

In [2]:
from remotemanager import URL

connection = URL(host='localhost')

This example connection is simply pointed at `localhost`, however you may define a connection to a machine with address or IP:

`connection = URL(user='username', host='remote.connection.address')`

`connection = URL(user='username', host='192.168.123.456')`

.. note::
    The only requirement for `URL` to function is that you must be able to ssh into the remote machine without any additional prompts from the remote. For connection difficulties regarding permssions, see the [relevant section](../Introduction.html#Connecting-to-a-Remote-Machine) of the introduction.

### Running Commands

With the concept of this remote `connection`, we can excecute commands and (more importantly) our function on this machine.

For commands, url provides a `cmd` method, which will execute any strings given

In [3]:
#connection.raise_errors=False

In [4]:
connection.submitter='sh'

In [5]:
connection.cmd('echo "this command is executed on the remote"')

RuntimeError: received the following stderr: 
/bin/bash: /opt/intel/oneapi/intelpython/python3.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)


### Running Functions

For function execution, we require a `Dataset`. Think of this dataset as a container for your function, with calculations to be added later on.

Like `URL`, this can be imported directly from `remotemanager`

To create a dataset, the only requirement is a callable function object. You must pass this object to the Dataset

.. note::
    When passing a function to the dataset, do not call it within the assigment. For example, for our multiply function, we should pass `function=multiply` _not_ `function=multiply()`

Here we are additionally specifying the `local_dir` and the `remote_dir`, which tells the Dataset where to put all relevant files on the local and remote machines, respectively.

If it suits your workflow, you can additionally specify a `run_dir` when appending a run. This is an additional folder within `remote_dir` where the script will be executed from. Thus, any files created by your function will be placed here.

In [5]:
from remotemanager import Dataset

ds = Dataset(function=multiply,
             url=connection,
             local_dir='temp_local',
             remote_dir='temp_remote',block_reinit=True)

NameError: name 'multiply' is not defined

### Creating runs

As the dataset is simply a container for the function, it is essentially useless in this state. To get some use out of it, we must append some runs.

To do this we use the `Dataset.append_run()` method. This will take the arguments in `dict` format, and store them for later.

You may do this in any way you see fit, the important part is to pass a dictionary which contains all ncessary arguments for the running of your function:

In [19]:
runs = [[21, 2],
        [64, 8],
        [10, 7]]

for run in runs:
    
    a = run[0]
    b = run[1]
    
    arguments = {'a': a, 'b': b}
    
    ds.append_run(arguments=arguments)

appended run runner-0
appended run runner-1
appended run runner-2


### Running and Retrieving your results

Now we have created a dataset and appended some runs, we can launch the calculations. This is done via the Dataset.run() method

Once the runs have completed, you can retrieve your results with `ds.fetch_results()`, and access them via `ds.results` once this is done

.. note::
    Be aware that the `fetch_results` method does not return your results, simply stores them in the `results` property.

In [21]:
ds.run(raise_errors=False)

assessing run for runner dataset-62eb4971-runner-0... checks passed, running
assessing run for runner dataset-62eb4971-runner-1... checks passed, running
assessing run for runner dataset-62eb4971-runner-2... checks passed, running


RuntimeError: received the following stderr: 
/bin/bash: /opt/intel/oneapi/intelpython/python3.9/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [10]:
# fetch the results, this loads them into the ds.results property for later access
import time

time.sleep(3)
results = ds.fetch_results()
print(results)  # This will print None, as fetch_results does not return anything

checking remotely for finished runs
None


In [11]:
# access this property any time after the results have been fetched. 
# This prevents the dataset attempting to poll the remote each time

print(ds.results)

[None, None, None, None, None, None]
